In [1]:
pwd

'/home/wsuser/work'

# Inserting necessary libraries

In [2]:
import numpy as np#used for numerical analysis
import tensorflow #open source used for both ML and DL for computation
from tensorflow.keras.models import Sequential #it is a plain stack of layers
from tensorflow.keras import layers #A layer consists of a tensor-in tensor-out computation function
#Dense layer is the regular deeply connected neural network layer
from tensorflow.keras.layers import Dense,Flatten
#Faltten-used fot flattening the input or change the dimension
from tensorflow.keras.layers import Conv2D,MaxPooling2D #Convolutional layer
#MaxPooling2D-for downsampling the image
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [3]:
tensorflow.__version__

'2.3.0'

In [4]:
tensorflow.keras.__version__

'2.4.0'

# Image Data Augumentation

In [11]:
#setting parameter for Image Data agumentation to the training data
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
#Image Data agumentation to the testing data
test_datagen=ImageDataGenerator(rescale=1./255)

# Loading our data and performing Data Augumentation

In [12]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_d6c4aa54985d4755b8f33ab78c4fe1f0 = 'https://s3.us.cloud-object-storage.appdomain.cloud'
else:
    endpoint_d6c4aa54985d4755b8f33ab78c4fe1f0 = 'https://s3.private.us.cloud-object-storage.appdomain.cloud'

client_d6c4aa54985d4755b8f33ab78c4fe1f0 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='k1ICcSyubCBRkuMzExqnzzFWzxk_udmE35sGXYL5wkNc',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_d6c4aa54985d4755b8f33ab78c4fe1f0)

streaming_body_1 = client_d6c4aa54985d4755b8f33ab78c4fe1f0.get_object(Bucket='naturaldisaster-donotdelete-pr-aj3prbxphtojo0', Key='dataset.zip')['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [13]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [15]:
pwd

'/home/wsuser/work'

In [16]:
import os
filenames = os.listdir('/home/wsuser/work/dataset/train_set')

In [17]:
#performing data agumentation to train data
x_train = train_datagen.flow_from_directory(r'/home/wsuser/work/dataset/train_set',target_size=(64, 64),batch_size=5,
                                            color_mode='rgb',class_mode='categorical')
#performing data agumentation to test data
x_test = test_datagen.flow_from_directory(r'/home/wsuser/work/dataset/test_set',target_size=(64, 64),batch_size=5,
                                            color_mode='rgb',class_mode='categorical')

Found 742 images belonging to 4 classes.
Found 198 images belonging to 4 classes.


In [18]:
print(x_train.class_indices)#checking the number of classes

{'Cyclone': 0, 'Earthquake': 1, 'Flood': 2, 'Wildfire': 3}


In [19]:
print(x_test.class_indices)#checking the number of classes

{'Cyclone1': 0, 'Earthquake1': 1, 'Flood1': 2, 'Wildfire1': 3}


In [20]:
from collections import Counter as c
c(x_train .labels)

Counter({0: 220, 1: 156, 2: 198, 3: 168})

# Creating the Model

In [21]:
# Initializing the CNN
classifier = Sequential()

# First convolution layer and poolingo
classifier.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))
# Second convolution layer and pooling
classifier.add(Conv2D(32, (3, 3), activation='relu'))
# input_shape is going to be the pooled feature maps from the previous convolution layer
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))

# Flattening the layers
classifier.add(Flatten())

# Adding a fully connected layer
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dense(units=4, activation='softmax')) # softmax for more than 2

In [22]:
classifier.summary()#summary of our model

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 32)        9248      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 27, 27, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 11, 11, 32)        9248      
_________________________________________________________________
flatten (Flatten)            (None, 3872)              0

# Compiling the Model

In [23]:
# Compiling the CNN
# categorical_crossentropy for more than 2
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Fitting the Model

In [24]:
classifier.fit_generator(
        generator=x_train,steps_per_epoch = len(x_train),
        epochs=20, validation_data=x_test,validation_steps = len(x_test))# No of images in test set

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/20
149/149 [==============================] - 32s 217ms/step - loss: 1.1916 - accuracy: 0.4582 - val_loss: 1.0800 - val_accuracy: 0.5505
Epoch 2/20
149/149 [==============================] - 32s 213ms/step - loss: 1.0074 - accuracy: 0.5876 - val_loss: 1.1755 - val_accuracy: 0.5505
Epoch 3/20
149/149 [==============================] - 31s 211ms/step - loss: 0.8887 - accuracy: 0.6146 - val_loss: 0.9522 - val_accuracy: 0.6465
Epoch 4/20
149/149 [==============================] - 31s 210ms/step - loss: 0.8046 - accuracy: 0.6685 - val_loss: 0.8259 - val_accuracy: 0.6566
Epoch 5/20
149/149 [==============================] - 31s 208ms/step - loss: 0.7213 - accuracy: 0.7291 - val_loss: 1.2338 - val_accuracy: 0.5556
Epoch 6/20
149/149 [==============================] - 31s 210ms/step - loss: 0.6916 - accuracy: 0.7237 - val_loss: 0.7315 - val_accuracy: 0.7273
Epoch 7/20
149/149 [==============================] - 

# Saving the Model

In [25]:
# Save the model
classifier.save('disaster.h5')

In [32]:
!tar -zcvf natural-disaster-classifier_new.tgz disaster.h5

disaster.h5


In [33]:
ls -1

dataset/
disaster.h5
model-bw.json
natural-disaster-classifier_new.tgz


In [34]:
#To save our file in machine learning repository we have to import library
!pip install watson-machine-learning-client --upgrade

     |████████████████████████████████| 538 kB 26.7 MB/s eta 0:00:01


In [36]:
#Replace the credentials that you got from watson Machine Learning Service
from ibm_watson_machine_learning import APIClient
wml_credentials = {
                   "url":"https://us-south.ml.cloud.ibm.com",
                   "apikey":"msSVl1ItE8WsNE-qN6DCjR__9IvD8xCx3U4Py3Oq2-rk"
                  }
client=APIClient(wml_credentials)

In [37]:
client = APIClient(wml_credentials)

In [38]:
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    #print(space)
    return(next(item for item in space['resources'] if item['entity']["name"] == space_name)['metadata']['id'])

In [39]:
space_uid = guid_from_space_name(client, 'naturaldisaster1')
print("Space UID = " + space_uid)

Space UID = 69d9ecd9-c6a9-48d3-856e-baf6ea02a14d


In [40]:
client.set.default_space(space_uid)

'SUCCESS'

In [41]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
scikit-learn_0.22-py3.6        154010fa-5b3b-4ac1-82af-4d5ee5abbc85  base
default_r3.6                   1b70aec3-ab34-4b87-8aa0-a4a3c8296a36  base
pytorch-onnx_1.3-py3.6         1bc6029

In [42]:
software_spec_uid = client.software_specifications.get_uid_by_name("tensorflow_1.15-py3.6")
software_spec_uid

'2b73a275-7cbf-420b-a912-eae7f436e0bc'

In [50]:
model_details = client.repository.store_model(model='natural-disaster-classifier_new.tgz',meta_props={
client.repository.ModelMetaNames.NAME:"CNN",
client.repository.ModelMetaNames.TYPE:"keras_2.2.4",
client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid}
                                             )
model_id = client.repository.get_model_uid(model_details)

Note: Warnings!! :  Model type keras_2.2.4 is deprecated. We recommend you use a supported model type. See Supported Frameworks https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/pm_service_supported_frameworks.html


In [51]:
model_id

'cf70fd82-d094-4cfa-b7b5-f876d3db13f3'

In [53]:
client.repository.download(model_id, 'my_model1.tar.gz')

Successfully saved model content to file: 'my_model1.tar.gz'


'/home/wsuser/work/my_model1.tar.gz'

In [54]:
model_json = classifier.to_json()
with open("model-bw.json", "w") as json_file:
    json_file.write(model_json)

# Predicting Results

In [55]:
from tensorflow.keras.models import load_model
from keras.preprocessing import image
model = load_model("disaster.h5") #loading the model for testing

In [61]:

streaming_body_2 = client_d6c4aa54985d4755b8f33ab78c4fe1f0.get_object(Bucket='naturaldisaster-donotdelete-pr-aj3prbxphtojo0', Key='cyclone.jpg')['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [65]:

streaming_body_3 = client_d6c4aa54985d4755b8f33ab78c4fe1f0.get_object(Bucket='naturaldisaster-donotdelete-pr-aj3prbxphtojo0', Key='cyclone.jpg')['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/
img = image.load_img(streaming_body_3,target_size = (64,64))#loading of the image\n

TypeError: expected str, bytes or os.PathLike object, not StreamingBody

In [66]:
import numpy as np
x = image.img_to_array(img)
x = np.expand_dims(x,axis = 0)

In [68]:
pred = model.predict_classes(x)

In [69]:
pred[0]

0

In [70]:
index=['Cyclone','Earthquake','Flood','Wildfire']
result=str(index[pred[0]])
result

'Cyclone'

In [22]:
!pip install jupyterthemes as jt

  Using cached jupyterthemes-0.20.0-py2.py3-none-any.whl (7.0 MB)


ERROR: Could not find a version that satisfies the requirement as
ERROR: No matching distribution found for as


In [23]:
!jt -t monokai

'jt' is not recognized as an internal or external command,
operable program or batch file.
